In [5]:
! install transformers tokenizers datasets evaluate accelerate

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.0.1 -> 25.0.1
[notice] To update, run: python3 -m pip install --upgrade pip


In [13]:
# Our dataset - 
urls = [
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0039.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0040.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0050.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0060.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0070.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0080.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_0090.shtml",
    "http://az.lib.ru/t/tolstoj_lew_nikolaewich/text_1860_dekabristy.shtml",
]

In [15]:
import html
import re
import requests

# Выполняет http GET-запрос по url и возвращает текст ответа
def download(url):
    return requests.get(url).text

striptags_re = re.compile(r"(<!--.*?-->|<[^>]*>)")
entity_re = re.compile(r"&([^;]+);")

# Удаляет html-теги с помощи регулярки (принимает строку)
def to_text(s):
    return html.unescape(striptags_re.sub("", s))

# Разбивает текст на строки, удаляет пустые строки и лишние пробелы. Ищет строку --> (конец HTML-комментария) и обрезает текст, начиная со следующей строки.
def beautify(s):
    lines = [x.strip() for x in s.split("\n") if x.strip() != ""]
    for i in range(min(100, len(lines))):
        if lines[i] == "-->":
            break
    return "\n".join(lines[i + 1 :] if i < 100 else lines)

# Создает файл dataset.txt для записи. Содержит  большой корпус текстов Льва Толстого.
with open("dataset.txt", "w", encoding="utf-8") as f:
    for u in urls:
        text = beautify(to_text(download(u)))
        f.write(text + "\n\n")

In [ ]:
!:bash
pragma dataset init mytext --size 1Gb
cp dataset.txt /home/jupyter/mnt/datasets/dataset.txt

In [2]:
import tokenizers as tok
import transformers as tr

# Применяем UNK для неизвестных слов
tokenizer = tok.Tokenizer(tok.models.BPE(unk_token="[UNK]"))
# Разбивает текст на токены по пробелам
tokenizer.pre_tokenizer = tok.pre_tokenizers.Whitespace()
# Применяем PAD для дополнения последовательности до определённой длины
trainer = tok.trainers.BpeTrainer(special_tokens=["[PAD]"])
tokenizer.train(["dataset.txt"], trainer)
# Включает режим дополнения, можно указать длину последовательностей токенов
tokenizer.enable_padding()

In [52]:
text = "Наглядно видим, как работает токенизация, также можем узнать их идентификаторы, но есть минусы - Последнее время мне стало жить тяжело. Я вижу, я стал понимать слишком много."
tokens = tokenizer.encode(text).tokens
ids = tokenizer.encode(text).ids
print(f'{text}\nСнизу представлена пара token:id')
for i, j in zip(tokens,ids):
    print(f'{i}:{j}')

Наглядно видим, как работает токенизация, также можем узнать их идентификаторы, но есть минусы - Последнее время мне стало жить тяжело. Я вижу, я стал понимать слишком много.
Снизу представлена пара token:id
На:328
гля:352
дно:320
видим:7730
,:8
как:250
работает:16456
токе:28561
ни:183
зация:22302
,:8
также:3779
можем:4748
узнать:3143
их:351
и:146
ден:1549
ти:225
фи:401
като:12686
ры:249
,:8
но:177
есть:547
мину:695
сы:424
-:9
Последнее:21696
время:504
мне:358
стало:1238
жить:1034
тяжело:1495
.:10
Я:137
вижу:2385
,:8
я:169
стал:693
понимать:3282
слишком:1692
много:1025
.:10
